#Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from abc import abstractmethod
import math
from collections import Counter
from typing import List
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

print("x_train: ", x_train)
print("x_test: ", x_test)
print("y_train: ", y_train)
print("y_test: ", y_test)

x_train:  [[6.5 3.  5.8 2.2]
 [5.5 3.5 1.3 0.2]
 [4.3 3.  1.1 0.1]
 [6.1 2.9 4.7 1.4]
 [4.8 3.  1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [6.3 2.8 5.1 1.5]
 [4.8 3.4 1.9 0.2]
 [6.1 3.  4.9 1.8]
 [5.1 3.8 1.6 0.2]
 [5.4 3.4 1.7 0.2]
 [5.4 3.4 1.5 0.4]
 [5.6 2.8 4.9 2. ]
 [7.7 3.8 6.7 2.2]
 [5.  3.6 1.4 0.2]
 [7.4 2.8 6.1 1.9]
 [6.  2.2 5.  1.5]
 [4.7 3.2 1.6 0.2]
 [5.1 3.5 1.4 0.2]
 [6.  2.2 4.  1. ]
 [5.  2.3 3.3 1. ]
 [7.9 3.8 6.4 2. ]
 [5.4 3.9 1.7 0.4]
 [5.4 3.9 1.3 0.4]
 [5.8 2.7 3.9 1.2]
 [5.  2.  3.5 1. ]
 [5.  3.2 1.2 0.2]
 [6.8 3.2 5.9 2.3]
 [6.7 3.  5.2 2.3]
 [5.8 2.7 5.1 1.9]
 [5.8 2.8 5.1 2.4]
 [6.3 3.4 5.6 2.4]
 [5.5 2.3 4.  1.3]
 [5.1 3.8 1.5 0.3]
 [4.4 3.  1.3 0.2]
 [6.5 3.2 5.1 2. ]
 [5.1 3.3 1.7 0.5]
 [4.9 3.1 1.5 0.1]
 [6.7 3.1 4.7 1.5]
 [6.1 3.  4.6 1.4]
 [5.5 2.5 4.  1.3]
 [5.7 2.6 3.5 1. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.1 4.4 1.4]
 [6.4 3.2 5.3 2.3]
 [4.5 2.3 1.3 0.3]
 [6.7 3.3 5.7 2.1]
 [5.7 3.  4.2 1.2]
 [5.1 3.7 1.5 0.4]
 [4.8 3.4 1.6 0.2]
 [6.3 2.9 5.6 1.8]
 [6.4 2.9 4.3 1.3]
 [

In [ ]:
@abstractmethod
class BayesSkeleton:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def _calc_priors(self, train_classes):
        for c in train_classes:
            self.priors[c] = self.priors.get(c, 0) + 1 / self.num_classes

    def _calc_likelihoods(self, train_features, train_classes):
        for c in self.classes:
            self.likelihoods[c] = {}
            for i, feature in enumerate(train_features.T):
                counter = Counter(feature[train_classes == c])
                for value, count in counter.items():
                    self.likelihoods[c][i, value] = count

In [ ]:
class NaiveBayes(BayesSkeleton):
    def __init__(self):
        self.priors = {}        # P(Y)
        self.likelihoods = {}   # P(X = {x1, x2, x3} | Y)

        self.classes: List[int] = None
        self.num_classes: int = None

    def build_classifier(self, train_features, train_classes):
        # Calculate the mean, var and prior probability of each class
        self.classes = np.unique(train_classes)
        self.num_classes = len(self.classes)

        self._calc_priors(train_classes)
        self._calc_likelihoods(train_features, train_classes)

    @staticmethod
    def data_discretization(data):
        bins = np.linspace(min(data), max(data), 4)
        discretized_data = np.digitize(data, bins, right=False) - 1
        return discretized_data

    def predict(self, sample):
        pass



class GaussianNaiveBayes(BayesSkeleton):
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

        self.classes: List[int] = None
        self.num_classes: int = None

        self._means: np.ndarray = []
        self._std = None

    def build_classifier(self, train_features, train_classes):
        self.classes = np.unique(train_classes)
        self.num_classes = len(self.classes)

        self._calc_priors(train_classes)
        self._calc_likelihoods(train_features, train_classes)

    @staticmethod
    def normal_dist(x, mean, std):
        std_part = 1 / (np.sqrt(2 * np.pi) * std)
        exp_part = np.exp( -((x - mean) ** 2) / (2 * (std ** 2)))
        return std_part * exp_part

    def predict(self, sample):
        pass
